Calcualting the stellar wind

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
import astropy.units as u
import astropy.constants as const
from scipy.interpolate import interpolate
from craftroom import resample
from astropy.convolution import convolve, Box1DKernel
from scipy.io.idl import readsav
from astropy import constants as const

#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20


In [2]:
rates = Table.read('tables/pceb_fit_results_rates.csv')
rates

Target,Teff,e_Teff,logg,e_logg,si,si_e,c,c_e,o,o_e,s,s_e,mass,q,tdiff_si,tdiff_c,tdiff_o,tdiff_s,Fsi,Fsi_e,Fc,Fc_e,Fo,Fo_e,Fs,Fs_e,porb_min,spt,mass_est,rad_est
str11,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str4,float64,float64
V-UZ-SEX,17146,15,7.801,0.007,-7.08,0.03,-6.21,0.05,-6.44,0.07,-7.36,0.3,0.511,-16.343,-1.997,-2.272,-2.135,-2.272,336385.0913,15320.37649,2008822.103,152446.5769,1149357.541,122104.1368,379599.6727,172820.6755,860.0,M4,0.22,0.258
V-LM-COM,28878,1122,7.358,0.1,-6.96,0.04,-5.9,0.04,-5.63,0.0,-6.96,0.0,0.424,-15.4,-0.913,-1.069,-1.024,-1.069,265938.7946,19504.52052,1870193.033,137163.9607,4182154.821,0.0,434781.2289,0.0,373.0,M3,0.36,0.369
WD-1339+606,37074,76,7.449,0.005,-7.88,0.4,-8.32,0.1,-8.05,0.4,-7.34,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356.0,M3,0.36,0.369
WD-1504+546,21730,53,7.67,0.01,-7.13,0.2,-8.25,0.09,-8.15,0.3,-7.43,0.6,0.464,-15.894,-1.515,-1.609,-1.758,-1.609,252309.6261,76582.37196,10163.38924,1388.40783,24018.85112,10935.24372,179235.037,163201.1191,1340.0,M3,0.36,0.369
WD-2317+268,28403,19,7.585,0.005,-6.78,0.03,-5.82,0.03,-4.66,0.1,-6.65,0.2,0.452,-15.541,-1.068,-1.232,-1.195,-1.232,443153.5923,20179.70829,2521500.235,114820.5499,44583348.57,6765937.313,995504.8128,302149.8672,1140.0,M3,0.36,0.369
WD-1436-216,22814,38,7.658,0.01,-7.19,0.05,-6.27,0.1,-5.92,0.4,-8.18,0.1,0.463,-15.83,-1.453,-1.548,-1.699,-1.548,220291.0386,16721.78698,975217.6868,148009.902,4117325.866,2499337.128,32024.76269,4860.434806,2910.0,M2,0.44,0.434
WD-1458+171,21406,4,7.445,0.0003,-6.19,0.3,-5.09,0.04,-5.96,0.0,-6.51,0.0,0.399,-15.65,-1.25,-1.344,-1.496,-1.344,1800481.797,819703.8918,12036313.03,730638.5581,3069072.482,0.0,1221455.993,0.0,237.0,M4,0.22,0.258
WD2257+162,22886,33,7.24,0.008,-7.38,0.05,-6.27,0.05,-8.03,0.5,-7.34,0.1,0.404,-15.587,-1.19,-1.288,-1.439,-1.288,118521.5178,8995.775491,818280.0127,62107.39973,26817.33484,20348.5671,185902.9555,28213.98152,464.0,M4.5,0.18,0.243
WD2256+249,21488,17,7.73,0.006,-6.88,0.02,-5.76,0.02,-5.72,0.05,-7.39,0.2,0.49,-15.996,-1.622,-1.719,-1.862,-1.719,479304.9558,14557.21339,3378443.483,102608.4168,6858518.74,520467.5137,211398.0657,64162.43974,250.0,M4,0.22,0.258


In [3]:
els = ['si', 'c', 'o', 's']
el_mass = [28.085, 12.011, 15.999, 32.06]
sol_frac = [6.649e-4, 2.365e-3,5.733e-3, 3.093e-4]

In [4]:
def mass_loss(mdot, Mwd, a, Rrd, Mrd): 
    """mass loss rate of mdwarf of mass Mrd, radius Rrd to produce an accretion rate mdot onto a white dwarf of mass Mwd at separation a"""
    m_loss = (((2*a*Mrd)/(Rrd*Mwd))**2) * mdot
   # m_loss = (mdot*(v**4)*(r**2))/((const.G.value**2)*(Mwd**2)*mf)
    return m_loss

def total_infer(mdot, el):
    els = np.genfromtxt('solar_abd.tsv', names=True, delimiter='\t', dtype=None, encoding=None)
    mf = els['mass_fraction'][np.where(els['Atom_X']==el)][0]
    total_mdot = mdot/mf
    return total_mdot

In [5]:
wind_rates = []
wind_errors = []
masses = []
for row in rates:
    if row['Target'] == 'WD0710+741':
    #if row['Teff'] < 30000:
        masses.append(row['mass_est'])
        mdots = []
        for i, el in enumerate(els):
            if row['{}_e'.format(el)] > 0.0:
                tot = row['F{}'.format(el)] / sol_frac[i]*(u.g/u.s)
                a = ((const.G*((row['mass']*u.M_sun).to(u.kg)+(row['mass_est']*u.M_sun).to(u.kg)) * ((row['porb_min']*u.min).to(u.s))**2)/(4*np.pi**2))**(1/3)
                print(a.to(u.R_sun))
                mdot = mass_loss(tot, row['mass'], a, (row['rad_est']*u.R_sun).to(u.m),  row['mass_est'])
                print(el, mdot)#.to(u.M_sun/u.yr))
                mdots.append(mdot.value)
        wind_rates.append(np.mean(np.array(mdots)))
        wind_errors.append(np.std(np.array(mdots)))

masses = np.array(masses)
print(len(masses))
#print(masses)

"""#print(wind_rates)
wind_rates = (np.array(wind_rates)*(u.g/u.s)).to(u.M_sun/u.yr).value
print(len(wind_rates))
print(wind_rates)
wind_errors = (np.array(wind_errors)*(u.g/u.s)).to(u.M_sun/u.yr).value
print(len(wind_errors))
#print(wind_errors)
mask = wind_rates < 1e-14
plt.errorbar(masses[mask], wind_rates[mask], yerr=wind_errors[mask], marker='o', ls='none')
#plt.errorbar(masses, wind_rates, yerr=wind_errors, marker='o', ls='none')
plt.xlabel('M dwarf Mass')
#plt.scatter(7, solar, marker='$\odot$', s= 500, color='k')
plt.axhline(2e-14, ls ='--', color='0.5', label='Solar')
#plt.xlim(0.1, 8.9)
plt.ylabel('\.M ($\mathrm{M}_{\odot}\mathrm{yr}^{-1}$)')
plt.yscale('log')
plt.tight_layout()
#plt.legend()
#plt.savefig('../talk/rates_state.png', dpi=150)
         """

0.7274562719928821 solRad
si 3696376834.48527 g / s
0.7274562719928821 solRad
c 11371196553.754877 g / s
0.7274562719928821 solRad
o 5281642979.722688 g / s
0.7274562719928821 solRad
s 14984482281.756517 g / s
1


"#print(wind_rates)\nwind_rates = (np.array(wind_rates)*(u.g/u.s)).to(u.M_sun/u.yr).value\nprint(len(wind_rates))\nprint(wind_rates)\nwind_errors = (np.array(wind_errors)*(u.g/u.s)).to(u.M_sun/u.yr).value\nprint(len(wind_errors))\n#print(wind_errors)\nmask = wind_rates < 1e-14\nplt.errorbar(masses[mask], wind_rates[mask], yerr=wind_errors[mask], marker='o', ls='none')\n#plt.errorbar(masses, wind_rates, yerr=wind_errors, marker='o', ls='none')\nplt.xlabel('M dwarf Mass')\n#plt.scatter(7, solar, marker='$\\odot$', s= 500, color='k')\nplt.axhline(2e-14, ls ='--', color='0.5', label='Solar')\n#plt.xlim(0.1, 8.9)\nplt.ylabel('\\.M ($\\mathrm{M}_{\\odot}\\mathrm{yr}^{-1}$)')\nplt.yscale('log')\nplt.tight_layout()\n#plt.legend()\n#plt.savefig('../talk/rates_state.png', dpi=150)\n         "

In [6]:
const.G

<<class 'astropy.constants.codata2018.CODATA2018'> name='Gravitational constant' value=6.6743e-11 uncertainty=1.5e-15 unit='m3 / (kg s2)' reference='CODATA 2018'>